In [5]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "round summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 5
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You invited Professor Dagger into the storage room, the space still cluttered despite your earlier attempt to clear some of the items out. The two of you began scouring through old furniture, leaving nothing unturned, yet, after two long hours, your hands held nothing but dust. You sensed the Professor’s eyes laced with disappointment, perhaps he has already given up the search, for to encounter a rare artifact like the one he described must require great deal of luck. But your desire for a better life fuels your persistence to hunt the fourth book down. You revisit the drawers you may have overlooked, potential compartments that are hidden behind intricate mechanisms like the seance table. And as you attempt to wrestle a large wooden shelf to another wall, your slip of the hand causes it to collide with the wall behind it, producing a hollow thud. Professor Dagger immediately placed his hands against the false wall, sliding across its surface gently like one would a body. Soon he sens

I select option 3. TURN: 1 


As you step cautiously through the dark corridor, the air thick with an unidentifiable stench, you can't help but comment on the surroundings. The walls, barely visible in the dim light, seem to pulse as if breathing, and the ground beneath your feet feels unnaturally soft, adding a layer of unease to your already heightened senses. Professor Dagger, a few steps ahead, pauses at your comment, his silhouette barely discernible.

"The walls of reality thin here," he whispers back, his voice barely above a murmur, "This place... it's not just a corridor. It's a threshold."

The corridor ends abruptly, opening into a wider, cavernous space that reeks of decay and ancient secrets. The sight that unfolds before you is both horrifying and mesmerizing. At the center of the room lies the near-withered body of the storage room's previously deceased owner's sister. Her face, a gaping cavity where something sinister writhes within. Surrounding her is a pentagram drawn

I select option 2. TURN: 2 


skill: spot_hidden
difficulty: medium

I am making a skill check using SPOT_HIDDEN against a difficulty of MEDIUM.
And I rolled a 10 for a result of SUCCESS. TURN: 3 


Your eyes carefully scan the withered body and the pentagram, your senses heightened by the eerie atmosphere of the hidden room. The success of your spot_hidden skill allows you to notice several key details that were not immediately apparent. The strange red fluid forming the pentagram seems to shimmer with an unnatural glow, suggesting it's not merely blood but something far more eldritch. You also notice that the symbols within the pentagram are of a language not known to the modern world, possibly hinting at ancient rites or summoning rituals.

Most disturbingly, as you examine the withered body, you see the writhing mass within the facial cavity more clearly. It's a nest of tentacles, each moving with a purpose, as if waiting for something—or someone—to come close enough. Despite the body's desiccated appearance, this activity within suggests a sinister

I select option 2. TURN: 4 


"Perhaps we should use something to touch or move the book from the podium," you suggest cautiously, aware of the potential danger that the pentagram and the withered body present. Professor Dagger nods in agreement, his eyes scanning the room for a suitable object.

After a brief search, you find a long, rusted iron rod among the debris scattered around the room. It seems sturdy enough to extend your reach without requiring you to step into the pentagram or get too close to the withered body.

With the rod in hand, you approach the podium carefully, making sure to maintain a safe distance from the pentagram. Your heart races as you extend the rod towards the book, the ancient tome seemingly emanating a palpable aura of foreboding.

possible actions:
1. Attempt to flip the book open with the rod.
2. Try to push the book off the podium using the rod.
3. Call out to the withered body, hoping for a reaction or a sign.

...

I am making a skill check using FIGHTING against a difficulty of HARD.
And I rolled a 55 for a result of FAILURE. TURN: 15 


Emboldened by your previous success, you attempt to strike the creature again, aiming to incapacitate it or at least fend it off long enough to make your escape. However, this time, as you swing the crowbar with all your might, the creature, seemingly learning from the last encounter, reacts with terrifying speed.

Its tentacles lash out, intercepting the crowbar mid-swing, and with a strength that belies its withered appearance, the creature wrenches the weapon from your grasp. The crowbar clatters to the ground, leaving you defenseless.

The failure of your attack leaves you exposed and vulnerable as the creature, now enraged, advances with renewed vigor. Its tentacles, moving with a purpose, reach out for you, intent on pulling you into its deadly embrace.

In this moment of desperation, with options dwindling and the creature closing in, you realize the grav

I select option 1. TURN: 16 


skill: dexterity
difficulty: hard

I am making a skill check using DEXTERITY against a difficulty of HARD.
And I rolled a 52 for a result of FAILURE. TURN: 17 


In a desperate bid for survival, you attempt to dodge past the creature to retrieve your only means of defense, the crowbar. However, the creature's tentacles, swift and unyielding, predict and counter your every move. Your attempt to outmaneuver it fails as one of the tentacles wraps around your ankle, pulling you off balance and to the ground with a jarring thud.

The cold, hard floor of the storage facility meets your back as you struggle against the creature's grip. The realization that you cannot reach the crowbar, now several feet away, sinks in. The creature looms over you, its tentacles poised to strike or drag you into its nightmarish embrace.

With the crowbar out of reach and the creature closing in, your situation appears dire. The failure to retrieve your weapon has left you vulnerable and at the mercy of the horror that now seeks to claim you.

po

I select option 1. TURN: 18 


skill: fighting
difficulty: hard

I am making a skill check using FIGHTING against a difficulty of HARD.
And I rolled a 40 for a result of FAILURE. TURN: 19 


In a final, desperate attempt to fend off the creature, you muster all your strength to kick at the tentacle that has ensnared your ankle. Your foot connects, but the impact feels like striking a mass of tightly coiled steel cables rather than flesh. The creature's grip does not waver; instead, it tightens, the pressure excruciating as it begins to drag you closer to its gaping maw.

Your efforts to fight back, to somehow turn the tide in this nightmarish confrontation, falter as the realization sets in: the creature's strength far surpasses your own. The failure of your attempt leaves you lying on the cold floor, breathless and defenseless, as the creature looms over you, its tentacles ready to strike.

In these final moments, as the creature prepares to deliver its fatal blow, the harsh reality of your situation becomes painfully clear. The storage facility, w